In [2]:
import pandas as pd
import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from flask import Flask,request,jsonify,render_template
import numpy as np

In [3]:
input_dict={
                'age': [34,42],
                'workclass':['State-gov','Self-emp-not-inc'],
                'fnlwgt':[1111,2323],
                'education-num':[12,11],
                'occupation':['Adm-clerical', 'Exec-managerial'],
                'relationship':['Husband', 'Wife'],
                'marital-status':['Separated','Separated'],
                'race':['Black','Asian-Pac-Islander'],
                'sex': ['Male', 'Female'], 
                'capital-gain':[3000,2000],
                'capital-loss': [0,1000],
                'hours-per-week':[70,50],
                'country':['United-States', 'Cuba']
            }


In [4]:
#convert the data into a dataframe
import pandas as pd
input_data=pd.DataFrame(input_dict,index=[0,1])
input_data    

,age,workclass,fnlwgt,education-num,occupation,relationship,marital-status,race,sex,capital-gain,capital-loss,hours-per-week,country
0,34,State-gov,1111,12,Adm-clerical,Husband,Separated,Black,Male,3000,0,70,United-States
1,42,Self-emp-not-inc,2323,11,Exec-managerial,Wife,Separated,Asian-Pac-Islander,Female,2000,1000,50,Cuba


In [5]:
categ_features =input_data.select_dtypes(include='object').columns
print(categ_features)
num_features=input_data.select_dtypes(exclude='object').columns
print(num_features)

Index(['workclass', 'occupation', 'relationship', 'marital-status', 'race',
       'sex', 'country'],
      dtype='object')
Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')


In [6]:
import joblib 
# load the model ad cleaning methods 
one_hencoder=joblib.load('1hot_model')
scaler_model=joblib.load('scale_model')
model=joblib.load('final_model1')

In [7]:
on_1=one_hencoder.transform(input_data[categ_features]).toarray()


In [8]:
pd.DataFrame(on_1) 

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
onc=one_hencoder.named_transformers_['cat_encoding'].named_steps['onehot']
onc.get_feature_names_out()

array(['marital-status_Married-AF-spouse',
       'marital-status_Married-civ-spouse',
       'marital-status_Married-spouse-absent',
       'marital-status_Never-married', 'marital-status_Separated',
       'marital-status_Widowed', 'relationship_Not-in-family',
       'relationship_Other-relative', 'relationship_Own-child',
       'relationship_Unmarried', 'relationship_Wife',
       'race_Asian-Pac-Islander', 'race_Black', 'race_Other',
       'race_White', 'sex_Male', 'workclass_Local-gov',
       'workclass_Never-worked', 'workclass_Private',
       'workclass_Self-emp-inc', 'workclass_Self-emp-not-inc',
       'workclass_State-gov', 'workclass_Without-pay',
       'occupation_Armed-Forces', 'occupation_Craft-repair',
       'occupation_Exec-managerial', 'occupation_Farming-fishing',
       'occupation_Handlers-cleaners', 'occupation_Machine-op-inspct',
       'occupation_Other-service', 'occupation_Priv-house-serv',
       'occupation_Prof-specialty', 'occupation_Protective-serv'

In [10]:
feature_names=onc.get_feature_names_out()
feature_names

array(['marital-status_Married-AF-spouse',
       'marital-status_Married-civ-spouse',
       'marital-status_Married-spouse-absent',
       'marital-status_Never-married', 'marital-status_Separated',
       'marital-status_Widowed', 'relationship_Not-in-family',
       'relationship_Other-relative', 'relationship_Own-child',
       'relationship_Unmarried', 'relationship_Wife',
       'race_Asian-Pac-Islander', 'race_Black', 'race_Other',
       'race_White', 'sex_Male', 'workclass_Local-gov',
       'workclass_Never-worked', 'workclass_Private',
       'workclass_Self-emp-inc', 'workclass_Self-emp-not-inc',
       'workclass_State-gov', 'workclass_Without-pay',
       'occupation_Armed-Forces', 'occupation_Craft-repair',
       'occupation_Exec-managerial', 'occupation_Farming-fishing',
       'occupation_Handlers-cleaners', 'occupation_Machine-op-inspct',
       'occupation_Other-service', 'occupation_Priv-house-serv',
       'occupation_Prof-specialty', 'occupation_Protective-serv'

In [11]:
c=pd.DataFrame(on_1,columns=feature_names)

In [12]:
len(categ_features) #7
for col in categ_features:
    a=input_data[col].nunique()
    print(f'the {col} has {a}\n')

the workclass has 2

the occupation has 2

the relationship has 2

the marital-status has 1

the race has 2

the sex has 2

the country has 2



In [13]:
clean_data2=scaler_model.transform(input_data[num_features])
clean_data2_=pd.DataFrame()
clean_data2_=pd.DataFrame(clean_data2,columns=num_features)
print(clean_data2_)

        age    fnlwgt  education-num  capital-gain  capital-loss  \
0  0.232877 -0.007589       0.733333          0.03      0.000000   
1  0.342466 -0.006766       0.666667          0.02      0.229568   

   hours-per-week  
0        0.704082  
1        0.500000  


In [14]:
ph=pd.concat([clean_data2_,c],axis=1)
ph

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,...,country_Portugal,country_Puerto-Rico,country_Scotland,country_South,country_Taiwan,country_Thailand,country_Trinadad&Tobago,country_United-States,country_Vietnam,country_Yugoslavia
0,0.232877,-0.007589,0.733333,0.03,0.000000,0.704082,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.342466,-0.006766,0.666667,0.02,0.229568,0.500000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
#make prediction on the cleaned data
predicted_value=model.predict(ph)
print(predicted_value)

[0 0]


In [16]:
type(predicted_value)

numpy.ndarray